In [1]:
import sys, json
import pandas as pd
import re
from datetime import datetime
import numpy as np

In [2]:
def create_df_normalize(json_file):
    '''
        Requires : json file
        Return : a structured data frame
    '''
    data = []
    for line in open(f"data/{json_file}", 'r'):
        data.append(json.loads(line))
    df = pd.json_normalize(data, )
    return df

In [3]:
def rearrange_columns(df, df_name):
    '''
        Requires : a data frame, name of dataframe
        Return : an organized data frame
        
    '''
    # Cleaning the column names
    column_names = list(df.columns)
    for i, name in enumerate(column_names):
        if (len(re.sub("[\w]+." ,"", name))) >= 2:
            column_names[i] = "_".join(name.split(".$")[:2])
        elif "$" in name or len(re.sub("[\w]+." ,"", name)) < 2:
            column_names[i] = name.split(".")[0]
        else:
            column_names[i] = name
    df.columns = column_names
    # Reordering the columns
    
    id_index = column_names.index("_id")
    new_order = column_names[id_index:] + column_names[:id_index]
    df = df[new_order]
    
    # Changing the data types
    for name in df.columns:
        if ("date" in name.lower()) or ("login" in name.lower()):
            df[name] = pd.to_datetime(df[name].astype(int, errors='ignore')/1000, unit='s')
        else:
            df[name]
    df.to_csv(f"{df_name}.csv", index = False)
    return df

In [4]:
users_df = rearrange_columns(create_df_normalize("users.json"), "users")
brands_df = rearrange_columns(create_df_normalize("brands.json"), "brands")
receipts_df = rearrange_columns(create_df_normalize("receipts.json"), "receipts")

In [6]:
for i in range(len(receipts_df["rewardsReceiptItemList"])):
    receipts_df["rewardsReceiptItemList"][i] = receipts_df["rewardsReceiptItemList"][i][0]

KeyError: 0

In [7]:
temp_df = receipts_df["rewardsReceiptItemList"]

In [8]:
temp_df = temp_df.apply(pd.Series)

In [16]:
temp_df = temp_df[['barcode','description','finalPrice','itemPrice','partnerItemId',
 'pointsPayerId','quantityPurchased','rewardsGroup','rewardsProductPartnerId','targetPrice']]

In [17]:
temp_df

,barcode,description,finalPrice,itemPrice,partnerItemId,pointsPayerId,quantityPurchased,rewardsGroup,rewardsProductPartnerId,targetPrice
0,4011,ITEM NOT FOUND,26.00,26.00,1,NaN,5.0,NaN,NaN,NaN
1,4011,ITEM NOT FOUND,1,1,1,NaN,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
3,4011,ITEM NOT FOUND,28.00,28.00,1,NaN,4.0,NaN,NaN,NaN
4,4011,ITEM NOT FOUND,1,1,1,NaN,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
x = receipts_df.drop(["rewardsReceiptItemList"], axis=1)

In [19]:
receipts_cleaned_df = pd.concat([x,temp_df], axis=1)

In [20]:
receipts_cleaned_df.to_csv("data/receipts_updated.csv", index=False)

In [21]:
df_list = [users_df, brands_df, receipts_cleaned_df]

for df in df_list:
    print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   _id           495 non-null    object        
 1   createdDate   495 non-null    datetime64[ns]
 2   lastLogin     433 non-null    datetime64[ns]
 3   active        495 non-null    bool          
 4   role          495 non-null    object        
 5   signUpSource  447 non-null    object        
 6   state         439 non-null    object        
dtypes: bool(1), datetime64[ns](2), object(4)
memory usage: 23.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   _id           1167 non-null   object
 1   cpg_id        1167 non-null   object
 2   cpg           1167 non-null   object
 3   brandCode     933 non-null    object
 4   barcode  

In [22]:
for df in df_list:
    display(df.head())

,_id,createdDate,lastLogin,active,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.799999952,2021-01-03 15:25:37.858000040,True,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.799999952,2021-01-03 15:25:37.858000040,True,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.799999952,2021-01-03 15:25:37.858000040,True,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:30.553999901,2021-01-03 15:25:30.596999884,True,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.799999952,2021-01-03 15:25:37.858000040,True,consumer,Email,WI


,_id,cpg_id,cpg,brandCode,barcode,category,categoryCode,name,topBrand
0,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN,511111019862,Baking,BAKING,test brand @1612366101024,False
1,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS,511111519928,Beverages,BEVERAGES,Starbucks,False
2,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176,511111819905,Baking,BAKING,test brand @1612366146176,False
3,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051,511111519874,Baking,BAKING,test brand @1612366146051,False
4,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False


,_id,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,purchaseDate,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,...,barcode,description,finalPrice,itemPrice,partnerItemId,pointsPayerId,quantityPurchased,rewardsGroup,rewardsProductPartnerId,targetPrice
0,5ff1e1eb0a720f0523000575,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,2021-01-03 00:00:00,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,...,4011,ITEM NOT FOUND,26.00,26.00,1,NaN,5.0,NaN,NaN,NaN
1,5ff1e1bb0a720f052300056b,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,2021-01-02 15:24:43,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,...,4011,ITEM NOT FOUND,1,1,1,NaN,1.0,NaN,NaN,NaN
2,5ff1e1f10a720f052300057a,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,2021-01-03 00:00:00,5.0,All-receipts receipt bonus,5,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
3,5ff1e1ee0a7214ada100056f,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,2021-01-03 00:00:00,5.0,All-receipts receipt bonus,5.0,...,4011,ITEM NOT FOUND,28.00,28.00,1,NaN,4.0,NaN,NaN,NaN
4,5ff1e1d20a7214ada1000561,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,2021-01-02 15:25:06,5.0,All-receipts receipt bonus,5.0,...,4011,ITEM NOT FOUND,1,1,1,NaN,1.0,NaN,NaN,NaN


## Data Quality Issues 
- There are duplicated values in users table
- Missing values in all three tables
- Wrong data types